In [40]:
import base64
import requests
import json
import os
import pandas

token = '<token>'
instance = '<host>'

rest_endpoint = 'https://%s/frontend/rs/genestack/' % instance
app_endpoint = 'https://%s/frontend/endpoint/application/invoke/genestack/' % instance

request_headers = {'Genestack-API-Token': token,
                   'Accept': 'application/json',
                   'Content-Type': 'application/json'}

def get_samples(study):
    url = rest_endpoint + 'integrationCurator/default-released/omics/samples?studyQuery=%s' % study
    r = requests.get(url=url, headers=request_headers)
    return r.json()['data']

def update_sample(sample, data):
    url = rest_endpoint + 'sampleCurator/default-released/samples/%s' % sample
    r = requests.patch(url=url, headers=request_headers, data=json.dumps(data))

In [41]:
# Create a blank study with 4 samples (default)
study_name = 'Biodesix test study'
template_accession = 'GSF000065'
number_of_initial_samples = 10

url_authenticate = app_endpoint + 'signin/authenticateByApiToken'
url_new_study = app_endpoint + 'study-metainfo-editor/createStudy'

s = requests.Session()
session = s.post(url_authenticate, json=[token])
study = json.loads(s.post(url=url_new_study, json=[study_name, template_accession, number_of_initial_samples]).text)['result']
print(study)

GSF000831


In [42]:
# add more samples
number_of_new_blank_samples = 1001
samples_parent = 'GSF' + str(int(study[3:]) + 1).zfill(6)
print('samples_parent=', samples_parent)
url_add_samples = app_endpoint + 'study-metainfo-editor/addSamples'
print(url_add_samples)
sample_group_id = 'sampleGroup:{}'.format(samples_parent)
sample_group_id_base64 = base64.b64encode(sample_group_id.encode('ascii')).decode('ascii')
print('sample_group_id_base64=', sample_group_id_base64)
res=s.post(url=url_add_samples, json=[study, sample_group_id_base64, number_of_new_blank_samples])
print(res, res.text)

samples_parent= GSF000832
https://qa.magnum.genestack.com/frontend/endpoint/application/invoke/genestack/study-metainfo-editor/addSamples
sample_group_id_base64= c2FtcGxlR3JvdXA6R1NGMDAwODMy
<Response [200]> {"result":null,"log":[]}


In [45]:
# Get samples in a study
samples_metadata = get_samples(study)
samples = [sample['metadata']['genestack:accession'] for sample in samples_metadata]
print(len(samples), samples[0:20], '...', samples[-20:])

1011 ['GSF000833', 'GSF000834', 'GSF000835', 'GSF000836', 'GSF000837', 'GSF000838', 'GSF000839', 'GSF000840', 'GSF000841', 'GSF000842', 'GSF000844', 'GSF000845', 'GSF000846', 'GSF000847', 'GSF000848', 'GSF000849', 'GSF000850', 'GSF000851', 'GSF000852', 'GSF000853'] ... ['GSF001825', 'GSF001826', 'GSF001827', 'GSF001828', 'GSF001829', 'GSF001830', 'GSF001831', 'GSF001832', 'GSF001833', 'GSF001834', 'GSF001835', 'GSF001836', 'GSF001837', 'GSF001838', 'GSF001839', 'GSF001840', 'GSF001841', 'GSF001842', 'GSF001843', 'GSF001844']


In [ ]:
# Update sample metadata
for sample in samples:
    update_sample(sample, {'Organism':'Homo sapiens'})